In [2]:
import numpy as numpy
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [30]:
data = {
    "사용자": ["사용자A", "사용자B", "사용자C", "사용자D", "사용자E"],
    "기생충": [5, 4, 1, 0, 3],
    "부산행": [3, 4, 2, 0, 3],
    "태극기": [3, 0, 5, 4, 0],
    "도둑들": [1, 0, 4, 2, 4],
    "닥터스": [4, 2, 4, 3, 5],
    "범죄도": [2, 0, 4, 0, 0]
}

df = pd.DataFrame(data)
df = df.set_index("사용자")
df.head()

,기생충,부산행,태극기,도둑들,닥터스,범죄도
사용자,,,,,,
사용자A,5,3,3,1,4,2
사용자B,4,4,0,0,2,0
사용자C,1,2,5,4,4,4
사용자D,0,0,4,2,3,0
사용자E,3,3,0,4,5,0


In [31]:
item_user_df = df.T
item_user_df.head()

사용자,사용자A,사용자B,사용자C,사용자D,사용자E
기생충,5,4,1,0,3
부산행,3,4,2,0,3
태극기,3,0,5,4,0
도둑들,1,0,4,2,4
닥터스,4,2,4,3,5


In [32]:
item_similarity = cosine_similarity(item_user_df)
item_similarity_df = pd.DataFrame(item_similarity, index=df.columns, columns=df.columns)
item_similarity_df

,기생충,부산행,태극기,도둑들,닥터스,범죄도
기생충,1.000000,0.954053,0.396059,0.483430,0.786618,0.438357
부산행,0.954053,1.000000,0.435890,0.613388,0.833734,0.507833
태극기,0.396059,0.435890,1.000000,0.720735,0.743736,0.822192
도둑들,0.483430,0.613388,0.720735,1.000000,0.903874,0.661693
닥터스,0.786618,0.833734,0.743736,0.903874,1.000000,0.641427
범죄도,0.438357,0.507833,0.822192,0.661693,0.641427,1.000000


In [33]:
user = "사용자E"
user_rating = df.loc[user]
user_rating

기생충    3
부산행    3
태극기    0
도둑들    4
닥터스    5
범죄도    0
Name: 사용자E, dtype: int64

In [34]:
user_unrated_item = user_rating[user_rating==0].index.to_list()
user_unrated_item

['태극기', '범죄도']

In [35]:
result = {}
for item in user_unrated_item:
    
    # 현재 안본 아이템의 유사도 점수 가져오기
    sim_score = item_similarity_df[item]
    # print(sim_score)
    
    # 사용자가 평점을 부여한 아이템 목록 가져오기
    user_rated_item = user_rating[user_rating > 0].index.to_list()
    # print(user_rated_item)
    
    # 사용자가 평점을 부여한 아이템에 대한 유사도 점수 필터링
    sim_scores = sim_score[user_rated_item]
    # print(sim_scores)
    
    # 유저 레이팅 점수 중에서 0점인 것은 제외한 리스트 가져오기
    user_rating_filtered = user_rating[user_rated_item]
    
    # 유사도 점수와 사용자 평점을 곱한 값을 더하기
    total_sum = sum(sim_scores * user_rating_filtered)
    # print(total_sum)
    
    # 전체 유사도 점수의 합으로 나누기
    sim_sum = sum(sim_scores)
    
    recommended_score = total_sum / sim_sum
    # print(recommended_score)
    
    result[item] = recommended_score

print(result)

{'태극기': 3.9615866580276267, '범죄도': 3.8645080811962913}


In [36]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 생성
data = {
    "사용자": ["사용자A", "사용자B", "사용자C", "사용자D", "사용자E"],
    "기생충": [5, 4, 1, 0, 3],
    "부산행": [3, 4, 2, 0, 3],
    "태극기": [3, 0, 5, 4, 0],
    "도둑들": [1, 0, 4, 2, 4],
    "닥터스": [4, 2, 4, 3, 5],
    "범죄도": [2, 0, 4, 0, 0],
}
df = pd.DataFrame(data).set_index("사용자")
interaction_matrix = df.values  # 사용자-아이템 행렬
user_id = 4  # 사용자E
item_id = 2  # 태극기


# 사용자 기반 협업 필터링
user_similarity = cosine_similarity(interaction_matrix)

# 사용자 기반 점수 예측
similar_users = user_similarity[user_id]
user_ratings = interaction_matrix[:, item_id]

# 유사도가 0 이상이고, 해당 아이템을 평가한 사용자만 고려
valid_users = (user_ratings > 0)
user_based_score = np.dot(similar_users[valid_users], user_ratings[valid_users]) / np.sum(
    similar_users[valid_users]
)
print(user_based_score)


# 아이템 기반 협업 필터링
item_similarity = cosine_similarity(interaction_matrix.T)

# 아이템 기반 점수 예측
user_ratings = interaction_matrix[user_id, :]
similar_items = item_similarity[item_id]

# 유사도가 0 이상이고, 해당 사용자가 평가한 아이템만 고려
valid_items = (user_ratings > 0)
item_based_score = np.dot(similar_items[valid_items], user_ratings[valid_items]) / np.sum(
    similar_items[valid_items]
)
print(item_based_score)


3.941120704882322
3.9615866580276267


In [41]:
# grouplens 데이터로 실습 해보기
ratings_df = pd.read_csv("../../data/ratings.csv")
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [125]:
movies = pd.read_csv("../../data/movies.csv")
movies.head()

merged_df = pd.merge(ratings_df, movies, on="movieId")
merged_df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [133]:
# 각 movieId의 rating 개수 계산
movie_rating_counts = merged_df['movieId'].value_counts()

# rating이 10개 이상인 영화만 필터링
valid_movies = movie_rating_counts[movie_rating_counts >= 10].index

# 해당 영화만 필터링한 데이터프레임
filtered_df = merged_df[merged_df['movieId'].isin(valid_movies)]

In [135]:
n = len(ratings_df['userId'].unique())
m = len(ratings_df['movieId'].unique())
print("users:", n)
print("movies:", m)

print("titles:", len(merged_df['title'].unique()))
print("titles:", len(merged_df['movieId'].unique()))

print("titles:", len(filtered_df['title'].unique()))
print("titles:", len(filtered_df['movieId'].unique()))


users: 610
movies: 9724
titles: 9719
titles: 9724
titles: 2269
titles: 2269


In [137]:
# 사용자-아이템 상호작용 행렬 생성
interaction_matrix = filtered_df.pivot_table(
    index="userId",      # 사용자 ID를 행으로
    columns="title",   # 영화 ID를 열로
    values="rating",     # 평점을 값으로
    fill_value=0         # 평가되지 않은 항목은 0으로 채움
)

# 결과 확인
user_item_df = pd.DataFrame(interaction_matrix)
user_item_df

item_user_df = user_item_df.T
item_user_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
(500) Days of Summer (2009),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5
10 Cloverfield Lane (2016),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
"10,000 BC (2008)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [138]:
item_similarity = cosine_similarity(item_user_df)
item_similarity_df = pd.DataFrame(item_similarity, index=user_item_df.columns, columns=user_item_df.columns)
item_similarity_df

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),...,Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
title,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",1.000000,0.099735,0.000000,0.180401,0.036417,0.125357,0.254537,0.078676,0.033942,0.034303,...,0.045145,0.055071,0.155553,0.184692,0.140750,0.091918,0.038152,0.211467,0.089634,0.372876
(500) Days of Summer (2009),0.099735,1.000000,0.173843,0.326122,0.225255,0.206167,0.198959,0.222581,0.169654,0.233237,...,0.401963,0.208675,0.104340,0.450463,0.402251,0.306076,0.260308,0.097935,0.276512,0.169385
10 Cloverfield Lane (2016),0.000000,0.173843,1.000000,0.036935,0.133272,0.045497,0.022956,0.074102,0.000000,0.291018,...,0.262842,0.119092,0.000000,0.297371,0.271721,0.227363,0.341125,0.200462,0.121770,0.032083
10 Things I Hate About You (1999),0.180401,0.326122,0.036935,1.000000,0.276708,0.282549,0.270490,0.095586,0.132512,0.088857,...,0.280660,0.142059,0.169472,0.154749,0.227712,0.340590,0.110769,0.167085,0.176720,0.160426
"10,000 BC (2008)",0.036417,0.225255,0.133272,0.276708,1.000000,0.265326,0.155047,0.102822,0.000000,0.113494,...,0.281309,0.109188,0.117015,0.215190,0.274656,0.272574,0.126702,0.114869,0.226588,0.112168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zoolander (2001),0.091918,0.306076,0.227363,0.340590,0.272574,0.246544,0.328696,0.196745,0.062037,0.134348,...,0.338465,0.273801,0.038226,0.294500,0.357185,1.000000,0.165195,0.144717,0.371858,0.159466
Zootopia (2016),0.038152,0.260308,0.341125,0.110769,0.126702,0.110579,0.131859,0.119460,0.096690,0.254504,...,0.314808,0.184455,0.045615,0.257177,0.343138,0.165195,1.000000,0.086647,0.234282,0.064098
eXistenZ (1999),0.211467,0.097935,0.200462,0.167085,0.114869,0.094899,0.130010,0.053689,0.032119,0.155810,...,0.118802,0.055587,0.191358,0.184123,0.118236,0.144717,0.086647,1.000000,0.192259,0.170341


In [139]:
user = 1
user_rating = user_item_df.loc[user]
print(user_rating[:5])
user_unrated_item = user_rating[user_rating==0].index.to_list()
print(user_unrated_item[:5])


title
'burbs, The (1989)                   0.0
(500) Days of Summer (2009)          0.0
10 Cloverfield Lane (2016)           0.0
10 Things I Hate About You (1999)    0.0
10,000 BC (2008)                     0.0
Name: 1, dtype: float64
["'burbs, The (1989)", '(500) Days of Summer (2009)', '10 Cloverfield Lane (2016)', '10 Things I Hate About You (1999)', '10,000 BC (2008)']


In [140]:
result = {}
for item in user_unrated_item:
    
    # 현재 안본 아이템의 유사도 점수 가져오기
    sim_score = item_similarity_df[item]
    # print(sim_score)
    
    # 사용자가 평점을 부여한 아이템 목록 가져오기
    user_rated_item = user_rating[user_rating > 0].index.to_list()
    # print(user_rated_item)
    
    # 사용자가 평점을 부여한 아이템에 대한 유사도 점수 필터링
    sim_scores = sim_score[user_rated_item]
    # print(sim_scores)
    
    # 유저 레이팅 점수 중에서 0점인 것은 제외한 리스트 가져오기
    user_rating_filtered = user_rating[user_rated_item]
    
    # 유사도 점수와 사용자 평점을 곱한 값을 더하기
    total_sum = sum(sim_scores * user_rating_filtered)
    # print(total_sum)
    
    # 전체 유사도 점수의 합으로 나누기
    sim_sum = sum(sim_scores)
    if sim_sum == 0:
        continue
    recommended_score = total_sum / sim_sum
    # print(recommended_score)
    
    result[item] = recommended_score

print(result)

{"'burbs, The (1989)": 4.389062328933131, '(500) Days of Summer (2009)': 4.3842073167732725, '10 Cloverfield Lane (2016)': 4.4457973595024685, '10 Things I Hate About You (1999)': 4.380779198202115, '10,000 BC (2008)': 4.3981614740237545, '101 Dalmatians (1996)': 4.366838212831702, '101 Dalmatians (One Hundred and One Dalmatians) (1961)': 4.421935308434345, '12 Angry Men (1957)': 4.424884542992919, '12 Years a Slave (2013)': 4.416013539742871, '127 Hours (2010)': 4.396206980298517, '13 Going on 30 (2004)': 4.378968099049052, '1408 (2007)': 4.4148553068918925, '15 Minutes (2001)': 4.36575219081793, '17 Again (2009)': 4.408802175500218, '1984 (Nineteen Eighty-Four) (1984)': 4.3584284927123536, '2 Days in the Valley (1996)': 4.3820159951707005, '2 Fast 2 Furious (Fast and the Furious 2, The) (2003)': 4.372229457850647, '20,000 Leagues Under the Sea (1954)': 4.375280195505085, '2001: A Space Odyssey (1968)': 4.40427489089277, '2010: The Year We Make Contact (1984)': 4.375904748064545, '201

In [141]:
sorted(list(result.items()), key=lambda x: x[1], reverse=True)

[('Coco (2017)', 4.528041661082443),
 ('Moana (2016)', 4.507102886778731),
 ('Rise of the Guardians (2012)', 4.49509005145931),
 ('Wild Tales (2014)', 4.480123099020783),
 ('Perks of Being a Wallflower, The (2012)', 4.479606458045887),
 ('Bicycle Thieves (a.k.a. The Bicycle Thief) (a.k.a. The Bicycle Thieves) (Ladri di biciclette) (1948)',
  4.472080957536399),
 ('Hunt, The (Jagten) (2012)', 4.468529445792911),
 ('Maleficent (2014)', 4.465676523048722),
 ('Parent Trap, The (1961)', 4.465226343676644),
 ('Secrets & Lies (1996)', 4.465077097585532),
 ('Mary and Max (2009)', 4.462083500292408),
 ('Mission: Impossible - Rogue Nation (2015)', 4.459867659789064),
 ('Double Indemnity (1944)', 4.459768833620537),
 ('Network (1976)', 4.458922505414809),
 ('Fantastic Mr. Fox (2009)', 4.458575526161346),
 ('Louis C.K.: Live at the Beacon Theater (2011)', 4.457974294797671),
 ('Bedtime Stories (2008)', 4.456737770316815),
 ('Dark Knight Rises, The (2012)', 4.456314018202457),
 ('Rope (1948)', 4.45